# The label_regions script

In [ ]:
#!/usr/bin/env python

'''
This script generates labels for given regions based the peaks files
downloaded from ENCODE project. 

Following is one example of how the script may be called to prepare 
the TF binding data from three experiments ENCSR000AKB, ENCSR000BJE, 
and ENCSR000DZL on cell line GM12878 for the data processing pipeline 
that involves deep learning with momma_dragonn, deepLIFT analysis, 
and MoDisCO motif analysis.

/home/ktian/kundajelab/tfnet/scripts/label_regions \
--positives \
/home/ktian/kundajelab/tfnet/ENCODE_data/GM12878-CTCF-human-ENCSR000AKB-optimal_idr.narrowPeak.gz,\
/home/ktian/kundajelab/tfnet/ENCODE_data/GM12878-SIX5-human-ENCSR000BJE-optimal_idr.narrowPeak.gz,\
/home/ktian/kundajelab/tfnet/ENCODE_data/GM12878-ZNF143-human-ENCSR000DZL-optimal_idr.narrowPeak.gz \
 \
--ambiguous ./_tmp_tuJQRK/_tmp_GM12878-CTCF-human-ENCSR000AKB-merged.narrowPeak.gz,\
./_tmp_tuJQRK/_tmp_GM12878-SIX5-human-ENCSR000BJE-merged.narrowPeak.gz,\
./_tmp_tuJQRK/_tmp_GM12878-ZNF143-human-ENCSR000DZL-merged.narrowPeak.gz\
 \
--background /home/ktian/kundajelab/tfnet/genome/hg19.tsv \
--prefix label  --stride 20 \

Arguments:
----------
Most of the syntax is borrowed from tfdragonn label-regions to
maintain compatibility. The difference is that this script 
uses native bedtools instead of PyBedtools 

--positives : gives a list of positive narrow peak files separated 
    by comma(','). Each of the positive peak file comes from
    an experiment and will define a task in the subsequent training 
    and analysis. In this example, three positive peak files were 
    given.

--ambiguous : gives ambiguous peaks files, separatedby comma(','), 
    one for each experiment. The positive and ambiguous peak files 
    from the same experiment must appear in the same order. 
    If any experiment has more than one ambiguous peak files, 
    they must be merged to produce a single ambiguous peak file. If any
    experiment has no ambiguous peak file, use '/dev/null' in its place.
    In this example, three merged ambiguous peak files were given.

--background : specifys the background region. In this example, 
    we specified hg19.tsv which gives the whole genome as the 
    background. Following is the beginning of hg19.tsv file.
    
    $ head hg19.tsv 
    chr1	1	249250621
    chr10	1	135534747
    chr11	1	135006516
    chr12	1	133851895
    chr13	1	115169878
    ...

--prefix : specifies the prefix of the output label file. In this example,
    the prefix 'label' is given, so the overall file name will be 
    label.intervals_file.tsv.gz
    
--stride : the stride used when making sliding windows, 20 is given
    in this example.
    
There are other possible arguments defined below. 
'''

In [ ]:
import argparse

def parse_args(args = None):
    '''
    argument parsing borrowed from tf-dragonn labelregions.
    '''
    parser = argparse.ArgumentParser('label_regions',
                                     description='Generate fixed length regions and their'
                                     ' labels for each dataset.',
                                     formatter_class=argparse.RawTextHelpFormatter)
    #parser.add_argument('--raw-intervals-config-file', type=os.path.abspath, default=None,
    #                    help='Includes task names and a map from dataset id -> raw interval file')
    parser.add_argument('--positives', type=str, default=None, help='positive bed files for each task, separated by commas.')
    parser.add_argument('--ambiguous', type=str, default=None, help='ambiguous bed files for each task, separated by commas.')
    parser.add_argument('--background', type=str, default=None, help='background bed file.')
    parser.add_argument('--prefix', type=str, required=True, help='prefix of output files')
    parser.add_argument('--n-jobs', type=int, default=1,
                        help='num of processes.\nDefault: 1.')
    parser.add_argument('--bin-size', type=int, default=200,
                        help='size of bins for labeling.\nDefault: 200.')
    parser.add_argument('--flank-size', type=int, default=400,
                        help='size of flanks around labeled bins.\nDefault: 400.')
    parser.add_argument('--stride', type=int, default=50,
                        help='spacing between consecutive bins.\nDefault: 50.')
    parser.add_argument('--genome', type=str, default='hg19',
                        help='Genome name.\nDefault: hg19.'
                        '\nOptions: hg18, hg38, mm9, mm10, dm3, dm6.')
    #parser.add_argument('--logdir', type=os.path.abspath,
    #                    help='Logging directory', default=None, required=False)
    args = parser.parse_args(args)
    return args


In [ ]:
'''
The directory layout is defined here, as relative path are used sometimes.

$TFNET_ROOT is set to the root of TFNET dir
 /-- scripts       such as label_region, prepare_data.py, run_deeplift.py, run_modisco.py, run_pipeline.py etc
 /-- genome        hg19.fa hg19.chrom.sizes hg19.tsv
 /-- ENCODE_data   peak files downloaded from ENCODE project
 /-- results       results
      |
      /-- nandi----/-- multi_tf --/-- multi_tf_nb_18_07_09/ working dirs down here
      |            /-- ZNF143
      |
      /-- nautilus-/-- CTCF
      |            /-- ZFX
      |
      /-- templates/-- config
'''
import os
from os.path import basename

ROOT_DIR   = os.getenv('TFNET_ROOT', "../../") 
scriptDir  = ROOT_DIR + "/scripts/"
dataDir    = ROOT_DIR + "/ENCODE_data/"
genomeDir  = ROOT_DIR + "/genome/"
resultsDir = "./"
logDir     = resultsDir + "log/"

In [2]:
'''
A typical debug message looks like the following:
2018-07-09 01:09:14 DEBUG processing DONE
'''
import logging

logging.basicConfig(
        format='%(asctime)s %(levelname)-5s %(message)s',
        level=logging.DEBUG,
        datefmt='%Y-%m-%d %H:%M:%S')


In [ ]:
'''
load the sizes of each chromosome so that we can ignore the first and last 5000 bp
'''
chrom_sizes = {}
def load_chrom_sizes(chrom_sizes_fn):
    global chrom_sizes
    chrom_sizes = {}
    with open(chrom_sizes_fn) as genome_fh:
        for line in genome_fh:
            chrom, size = line.split()
            chrom_sizes[chrom.strip()] = int(size)

# ignore the first and last 5000 bp of each chromosome 
min_dist_from_edge = 5000

In [3]:
def make_windows(in_name, out_name, bin_size = 200, stride = 50, expand = True):
    '''
        takes in regions and expands region lengths to be a multiple of stride (default 50)
        outputs a list of regions, result of sliding window of length bin_size (default 200)
    '''
    with open(in_name,'r') as tsvin, open(out_name, 'w') as tsvout:
        # for row in tsvin:
        for cnt, line in enumerate(tsvin):
            row = line.split()
            chrom = row[0]
            start = int(row[1])
            end   = int(row[2])

            if chrom[3] == 'M' or chrom[3] == 'm':
                continue
            if len(chrom) > 5:
                continue

            peak_length = end - start
            if expand :
                # total_length = stride * ceiling = smallest multiple of stride greater than peak_length
                total_length = stride * int((peak_length - bin_size + stride - 1)/stride) + bin_size
            else : 
                total_length = peak_length # no expand

            # extend_left = int((total_length - peak_length)/2) # extend on both sides
            extend_left = 0 # extand only on the right, this is tf-dragonn's style
            extend_right = total_length - peak_length - extend_left

            left  = start - extend_left
            right = end + extend_right

            # write out the new "unified" intervals
            right_max = chrom_sizes[chrom] - min_dist_from_edge
            for le in range(max(left, min_dist_from_edge), 
                            min(right, right_max) - bin_size + 1, stride):
                tsvout.write(chrom + "\t" + str(le) + "\t" + str(le + bin_size) + "\n")

In [4]:
import contextlib
@contextlib.contextmanager
def dummy_context_mgr():
    yield None

def label_regions_fast(region, positive, ambiguous, labels):
    '''
        label regions for one task
        O(n) to go through intervals in region file and label them
        assuming regions, positives, ambiguous files are all sorted

        Parameters
        ----------
        region:     input file with list of intervals to be labeled
        positive:   input file with list of positive intervals (subset of region)
        ambiguous:  input file with list of ambiguous intervals (subset of region)
        labels:     output file with one column of labels
    '''
    
    # Open the file with read only permit
    with open(region,'r') as reg, open(positive, 'r') as pos, open (labels, 'w') as lab:
        
        # conditional with block
        with open(ambiguous, 'r') if ambiguous != None else dummy_context_mgr() as amb:

            # read in p_line, r_line, and a_line from files
            p_line = pos.readline()
            r_line = reg.readline()
            if ambiguous != None:
                a_line = amb.readline()
                read_a = True
            else:
                a_line = None
                read_a = False

            # iterate through regions & determine labels
            while r_line:
                read_p = False # if p matched r and we should move to next positive interval
                read_a = False # if a matched r ...
                if r_line == p_line: # positive
                    out = 1
                    read_p = True
                    if r_line == a_line:
                        read_a = True
                elif r_line == a_line: # ambiguous
                    out = -1
                    read_a = True
                else: # negative
                    out = 0

                lab.write(str(out) + "\n")

                r_line = reg.readline()
                if read_p:
                    p_line = pos.readline()
                if read_a:
                    a_line = amb.readline()
# can be simplified if we use intersect -c


In [ ]:
def unzip_peaks(positives, ambiguous):
    '''
    # unzip all original positive  input interval files to tmpDir
    # unzip all original ambiguous input interval files to tmpDir
    '''

    ### positives
    pos_unzips = []
    for p_orig in positives:
        if p_orig[-3:] == ".gz":
            # basename extracts the last part of the path, the filename
            p_unzip = tmpDir + basename(p_orig[:-3]) # remove ".gz" from filename

            # unzip and sort p_orig to p_unzip
            os.system("pigz -d -c " + p_orig + " | bedtools sort > " + p_unzip) 
        else:
            p_unzip = tmpDir + basename(p_orig)
            # copy and sort p_orig to p_unzip
            os.system("cat "       + p_orig + " | bedtools sort > " + p_unzip) 
        pos_unzips.append(p_unzip)

    ### ambigous
    amb_unzips = None
    if ambiguous != None:
        assert(len(positives) == len(ambiguous)) # ensure same number of tasks
        
        amb_unzips = []
        for a_orig in ambiguous:
            if a_orig == None:
                a_unzip = ""
            else:
                if a_orig[-3:] == ".gz":
                    a_unzip = tmpDir + basename(a_orig[:-3]) # remove ".gz"
                    os.system("pigz -d -c " + a_orig + " | bedtools sort > " + a_unzip)
                else:
                    a_unzip = tmpDir + basename(a_orig)
                    os.system("cat "       + a_orig + " | bedtools sort > " + a_unzip)
            amb_unzips.append(a_unzip)

    return pos_unzips, amb_unzips

In [ ]:
def gen_intersects(pos_unzips, amb_unzips, tmp_regions):
    # generate intersects with positives
    pos_intersects = []
    for p in pos_unzips:
        p_intersect = tmpDir + "_tmp_pint_" + basename(p)
        os.system("bedtools intersect -a " + tmp_regions + " -b " + p + \
            " -sorted -wa -f 0.5 -F 0.5 -e -u > " + p_intersect) # add -c later
        #os.system("bedtools intersect -a " + tmp_core_regions + " -b " + p + \
        #    " -sorted -wa -f 0.5 -F 0.5 -e -c | cut -f 4 > " + p_intersect)
        pos_intersects.append(p_intersect)

    # generate intersects with ambiguous
    amb_intersects = []
    for a in amb_unzips:
        a_intersect = tmpDir + "_tmp_aint_" + basename(a)
        os.system("bedtools intersect -a " + tmp_regions + " -b " + a + \
            " -sorted -wa -f 0.5 -F 0.5 -e -u > " + a_intersect) # add -c later
        #os.system("bedtools intersect -a " + tmp_core_regions + " -b " + a + \
        #    " -sorted -wa -f 0.5 -F 0.5 -e -c | cut -f 4 > " + a_intersect)
        amb_intersects.append(a_intersect)

    return pos_intersects, amb_intersects

In [ ]:
def handle_background_intervals(regions, tmp_core_regions, chrom_sizes_fn, 
                                bin_size=200, flank_size=400):
    if regions == None:
        # if no background regions were specified, use the union of positives as regions
        tmp_regions = tmp_core_regions
    else:    # sort and merge the background regions
        tmp_regions = tmpDir + "_tmp_regions.tsv"
        cmd = "cat " + regions + " | cut -f 1-3 | bedtools sort | bedtools merge  > " + tmp_regions
        os.system(cmd)

        tmp_expanded_bkg_regions = tmpDir + "_tmp_expanded_bkg_regions.tsv"
        expanded_window_size = bin_size + 2 * flank_size

        # for background, make expanded sized windows directly (1000 instead of usual 200)
        make_windows(tmp_regions, tmp_expanded_bkg_regions, 
                     bin_size=expanded_window_size, stride=expanded_window_size)     

        # generate shrunk regions for labeling (using negative flank_size)
        tmp_bkg_regions = tmpDir + "_tmp_bkg_regions.tsv"
        os.system("bedtools slop -i " + tmp_expanded_bkg_regions + " -g " + \
            chrom_sizes_fn + " -b " + str(-flank_size) + " > " + tmp_bkg_regions)

        os.system("cat " + tmp_core_regions + " >> " + tmp_bkg_regions)

        cmd = "cat " + tmp_bkg_regions + " | bedtools sort | uniq -u > " + tmp_regions
        os.system(cmd)

    return tmp_regions

In [5]:
def label_regions_multitask(labels_multitask, chrom_sizes_fn, positives, 
                            ambiguous = None, regions = None,
                            bin_size=200, flank_size=400, stride=50,
                            n_jobs=1, genome='hg19',
                            min_bin_distance_to_chrom_edge=5000):
    ''' 
    generate labels for multitask training

    Parameters
    ----------
    labels_multitask : output labels file.
    chrom_sizes_fn   : file that contains the sizes of reference genomes.
    positives        : the true positives, a list of paths, one for each experiment,
                       mandatory.
    ambiguous        : the ambiguous set, a list of paths, one for each experiment,
                       optional. if present, must have the same length as the 
                       positives, so that we can match ambiguous to the positives
    regions          : (not supported yet) 
                       total regions being labeled. If not specified, 
                       use the union of positives
    '''
    #os.system("mkdir -p " + tmpDir)

    logging.debug("bin-size=%d, stride=%d" % (bin_size, stride))
    load_chrom_sizes(chrom_sizes_fn)

    pos_unzips, amb_unzips = unzip_peaks(positives, ambiguous)

    
    # sort and merge the positives 
    tmp_pos_regions = tmpDir + "_tmp_pos_regions.tsv"
    cmd = "cat " + " ".join(pos_unzips) + \
        " | cut -f 1-3 | bedtools sort | bedtools merge > " + tmp_pos_regions
    os.system(cmd)

    # unify the core region lengths to multiple of strides and make windows
    tmp_core_regions = tmpDir + "_tmp_core_regions.tsv"
    make_windows(tmp_pos_regions, tmp_core_regions, bin_size=bin_size, stride=stride)     

    tmp_regions = handle_background_intervals(regions, tmp_core_regions, chrom_sizes_fn, 
                                              bin_size=bin_size, flank_size=flank_size)

    logging.info("LABEL_REGIONS sort merge done")

    # generate positive and ambigous intersects
    pos_intersects, amb_intersects = gen_intersects(pos_unzips, amb_unzips, tmp_regions)

    # generate expanded regions for added context
    tmp_expanded_regions = tmpDir + "_tmp_expanded_regions.tsv"
    os.system("bedtools slop -i " + tmp_regions + " -g " + \
        chrom_sizes_fn + " -b " + str(flank_size) + " > " + tmp_expanded_regions)

    logging.info("LABEL_REGIONS intersect done")

    # label regions for each task and paste to the regions
    for i in range(len(positives)):
        tmp_labels = tmpDir + "_tmp_labels_" + str(i) + ".tsv"
        if ambiguous:
            a_intersect = amb_intersects[i]
        else:
            a_intersect = None
        label_regions_fast(tmp_regions, pos_intersects[i], a_intersect, tmp_labels)
        tmp_paste = tmpDir + "_tmp_paste"
        os.system("paste " + tmp_expanded_regions + " " + tmp_labels + " > " + tmp_paste)
        os.system("mv -f " + tmp_paste + " " + tmp_expanded_regions)

    logging.info("LABEL_REGIONS labels done")
    # shuffle the lines
    os.system("shuf " + tmp_expanded_regions + " | pigz > " + labels_multitask)

    logging.info("LABEL_REGIONS ALL DONE *****")

In [6]:

def run_label_regions_from_args():
    args = parse_args()
    if args.positives is not None: # and args.background is not None:
        positives = args.positives.split(',')
        if args.ambiguous is not None:
            ambiguous = args.ambiguous.split(',')
            assert len(ambiguous) == len(positives)
        else:
            ambiguous = None
        path_to_dataset_intervals_file = os.path.abspath("{}.intervals_file.tsv.gz".format(args.prefix))
        #print(path_to_dataset_intervals_file)
        label_regions_multitask(path_to_dataset_intervals_file,
                                genomeDir + "hg19.chrom.sizes", 
                                positives, ambiguous=ambiguous, regions=args.background, 
                                bin_size=args.bin_size, flank_size=args.flank_size, 
                                stride=args.stride, genome=args.genome, n_jobs=args.n_jobs)
    else:
        raise RuntimeError("Must pass in positives, background, and task names!")

In [ ]:
# guarantee to clean up tmp dir
import contextlib
import tempfile
import shutil

@contextlib.contextmanager
def make_temp_directory():
    temp_dir = tempfile.mkdtemp(dir = ".", prefix = "_tmp_")
    try:
        yield temp_dir
    finally:
        shutil.rmtree(temp_dir)

if __name__ == '__main__':
    with make_temp_directory() as temp_dir:
        global tmpDir
        tmpDir = temp_dir + "/"
        run_label_regions_from_args()